# PyData London 2022 - Man Group Charting Competition

Source: https://github.com/man-group/pydata2022

To run the notebook and serve the web app you would need to run

```bash
pip install pandas pyarrow panel hvplot holoviews 
```

We will end up building an interactive web app called *Options Analytics* that looks like below.

You can use the app to do "What if" Analysis and create trading ideas.

![Video](options-analytics-speedup.gif)

You can serve the app via `panel serve option-analytics.ipynb`.

## Imports

In [ ]:
import pandas as pd
import panel as pn
import hvplot.pandas
import holoviews as hv
from io import StringIO

## Configuration

In [ ]:
pn.config.raw_css.append("""
.bk-root {
  height: calc( 100vh - 150px ) !important;
}
""")

pn.extension("perspective", sizing_mode="stretch_width", template="fast")

In [ ]:
SITE = "Man-Group"
TITLE = "Options Analytics"
LOGO = "https://pydata.org/london2022/wp-content/uploads/2022/02/PyData_logo.png"

SOURCE_DATA = "https://github.com/man-group/pydata2022/raw/main/option_chain_data.parquet"
LOCAL_DATA = "option_chain_data.parquet"

COLUMNS = ["type", "strike", "inTheMoney", "impliedVolatility", "lastPrice", "lastTradeDate", "volume", "openInterest", "bid", "ask", "spread", "change", "percentChange", "contractSymbol", ]
HOVER_COLS = ["strike", "impliedVolatility", "lastPrice", "volume", "openInterest", "bid", "ask", "spread", "contractSymbol",]

## Data

### Extract

In [ ]:
def extract():
    try:
        return pd.read_parquet(LOCAL_DATA)
    except FileNotFoundError:
        data = pd.read_parquet(SOURCE_DATA)
        data.to_parquet(LOCAL_DATA)
        return data


source_data = extract()
source_data.head(2)

## Transform

In [ ]:
def transform(data: pd.DataFrame) -> pd.DataFrame:
    data["date"] = data["date"].astype("datetime64[ns]")
    data["inTheMoney"] = data["inTheMoney"].astype("bool")

    data["spread"] = data["ask"] - data["bid"]
    return data[COLUMNS]


data = transform(source_data)
data.head(2)

## Plot

In [ ]:
def get_plots(data: pd.DataFrame):
    call_data = data[data["type"] == "Call"]
    put_data = data[data["type"] == "Put"]
    plots = [
        call_data.hvplot(
            x="strike",
            y="impliedVolatility",
            responsive=True,
            min_height=300,
            ylabel="Implied Volatility",
            title="Call",
            hover_cols=HOVER_COLS,
        ),
        put_data.hvplot(
            x="strike",
            y="impliedVolatility",
            responsive=True,
            min_height=300,
            ylabel="Implied Volatility",
            title="Put",
            hover_cols=HOVER_COLS,
        ),
        call_data.hvplot(
            x="strike",
            y="openInterest",
            responsive=True,
            height=200,
            ylabel="Open Interest",
            hover_cols=HOVER_COLS,
        ),
        put_data.hvplot(
            x="strike",
            y="openInterest",
            responsive=True,
            height=200,
            ylabel="Open Interest",
            hover_cols=HOVER_COLS,
        ),
        call_data.hvplot(
            x="strike",
            y="spread",
            responsive=True,
            height=200,
            ylabel="Spread",
            hover_cols=HOVER_COLS,
        ),
        put_data.hvplot(
            x="strike",
            y="spread",
            responsive=True,
            height=200,
            ylabel="Spread",
            hover_cols=HOVER_COLS,
        ),
    ]
    layout = hv.Layout(plots).cols(2)
    return layout


plots = get_plots(data)
plots

## Widgets

In [ ]:
stock_selector = pn.widgets.Select(options=["Google"], name="Name", max_width=300)

In [ ]:
def get_stringio(data: pd.DataFrame) -> StringIO:
    sio = StringIO()
    data.to_csv(sio)
    sio.seek(0)
    return sio

sio_to_download = get_stringio(data)

download_button = pn.widgets.FileDownload(
    sio_to_download,
    embed=True,
    filename="google.csv",
    sizing_mode="fixed",
    width=150,
    height=52,
    name="Download",
    label="google.csv"
)

## Panels

In [ ]:
plot_pane = pn.pane.HoloViews(plots, sizing_mode="stretch_both")
pivot_pane = pn.pane.Perspective(data, sizing_mode="stretch_both")
doc_pane = pn.pane.Markdown("""
# Option Chain Visualization

The *Option Analytics* tool was developed as a part of the PyData London 2022 chart visualization competition by **Man-Group**. 

The data is *option chain data*. See [Investopedia - Option Chain](https://www.investopedia.com/terms/o/optionchain.asp#:~:text=An%20options%20chain%2C%20also%20known,within%20a%20given%20maturity%20period).

I've used [Panel](https://panel.holoviz.org/index.html) as *data app framework* and [hvPlot](https://hvplot.holoviz.org/). These tools are superior for working in an out of a notebook. Especially for quant analysis.

Source: [man-group/pydata2022](https://github.com/man-group/pydata2022)
""", name="🎓 Docs")

## Layouts

In [ ]:
tab_layout = pn.Tabs(("📈 Plot", plot_pane), ("🛠️ Pivot", pivot_pane), doc_pane, margin=10)

In [ ]:
tool_layout = pn.Column(
    pn.Row(stock_selector, download_button, margin=(10,0,20,0)),
    tab_layout,
)
tool_layout.servable()

Note: For some unknown reason the Perspective Viewer pivot table does not render in the notebook.

## Template

In [ ]:
template = pn.state.template

In [ ]:
template.param.update(
    site="Man-Group Viz Competition",
    title=TITLE,
    logo=LOGO,
)

In [ ]:
if template.theme == pn.template.DarkTheme:
    pivot_pane.theme = "material-dark"

## Serve the app

The app can now be served via `panel serve option-analytics.ipynb` and is available via [http://localhost:5006/option_smiles?theme=dark](http://localhost:5006/option_smiles?theme=dark).

![Video](options-analytics-speedup.gif)